In [ ]:
iteration = '10 Iter'
#saving.to_csv('data_cycles-40.csv') # for 6th and further increased hidden from 16 to 32

In [ ]:
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

import deepchem as dc
from deepchem.models.normalizing_flows import NormalizingFlow, NormalizingFlowModel
from deepchem.models.optimizers import Adam
from deepchem.data import NumpyDataset
from deepchem.splits import RandomSplitter
from deepchem.molnet import load_tox21

import rdkit
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw

from IPython.display import Image, display

import selfies as sf

import tensorflow as tf
print(tf.__version__)

import tensorflow_probability as tfp

tfd = tfp.distributions
tfb = tfp.bijectors

# "set the environment variable `TF_USE_LEGACY_KERAS=True` to "
# "configure TensorFlow to use `tf_keras` when accessing `tf.keras`." 
#ImportError: `keras.optimizers.legacy` is not supported in Keras 3. 
#When using `tf.keras`, to continue using a `tf.keras.optimizers.legacy`
#optimizer, you can install the `tf_keras` package (Keras 2) and set the environment variable
#TF_USE_LEGACY_KERAS=True` to configure TensorFlow to use `tf_keras` when accessing `tf.keras`.

import tf_keras as keras
tfk = keras
tfk.backend.set_floatx('float64')

In [ ]:
print(keras.__version__)

In [ ]:
import matplotlib.pyplot as plt
import sklearn.datasets as datasets
import pandas as pd
import numpy as np
import copy

import os
import sys
from rdkit.Chem import RDConfig
sys.path.append(os.path.join(RDConfig.RDContribDir, 'SA_Score'))
import sascorer
from rdkit import Chem
from rdkit.Chem import QED
from rdkit import RDLogger 
RDLogger.DisableLog('rdApp.*')  # suppress error messages

import math # just for train/test split
import time
import seaborn as sns
import matplotlib.pyplot as plt

from collections import OrderedDict

In [ ]:
def get_data(df):
  #data = df['len'].values.reshape((-1,1))
  data = df[['QED', 'len']]#.values.reshape((-1,1)) # to get multiple column context
  return data

def preprocess_smiles_NaN(smiles):
    global counter_failed
    try:
        encoded = sf.encoder(smiles)
    except:
        counter_failed =+ 1
        encoded = 'nop'    # substitute empty strings for [nop]
    return encoded

def preprocess_smiles(smiles):
    encoded = sf.encoder(smiles)
    return encoded

def keys_int(symbol_to_int):
  d={}
  i=0
  for key in symbol_to_int.keys():
    d[i]=key
    i+=1
  return d

def get_selfies(df):
    global selfies_list, largest_selfie_len, int_mol, selfies_alphabet, onehots, input_tensor, noise_tensor

    sf.set_semantic_constraints()  # reset constraints
    constraints = sf.get_semantic_constraints()
    constraints['?'] = 3
    sf.set_semantic_constraints(constraints)
    
    selfies_list = np.asanyarray(df.selfies)
    selfies_alphabet = sf.get_alphabet_from_selfies(selfies_list)
    # selfies_alphabet.add('.')  # test this
    selfies_alphabet.add('[nop]')  # Add the "no operation" symbol as a padding character; selfies_alphabet.remove('[P]')
    selfies_alphabet = list(sorted(selfies_alphabet))
    # selfies_list = np.asanyarray(df.selfies) # added this line for a subset only if I want to load more selfie alphabet examples
    largest_selfie_len = 40 #max(sf.len_selfies(s) for s in selfies_list)
    symbol_to_int = dict((c, i) for i, c in enumerate(selfies_alphabet))
    int_mol=keys_int(symbol_to_int)
    
    onehots=sf.batch_selfies_to_flat_hot(selfies_list, symbol_to_int, largest_selfie_len)
    input_tensor = tf.convert_to_tensor(onehots, dtype='float32')
    noise_tensor = tf.random.uniform(shape=input_tensor.shape, minval=0, maxval=1, dtype='float32')
    data = tf.add(input_tensor, noise_tensor) # dequantized data
    return data

In [ ]:
#df = pd.read_csv('../../Dataset_after_cycle_4-corrected.csv',delimiter=',').dropna(subset=['QED']).drop_duplicates(subset=['Original_SMILES'])
#data = df[df["Origin"] == 'Initial'] #df.copy()
data = pd.read_csv('data_cycles-40.csv',delimiter=',').dropna(subset=['QED']).drop_duplicates(subset=['Original_SMILES'])
data['len'] = data['selfies'].apply(lambda x: sf.len_selfies(x))

In [ ]:
x = np.float32(get_selfies(data))
y = np.float32(get_data(data))

index = math.floor(x.shape[0]*1.0)

x_train = x[:index]
y_train = y[:index,:]

In [ ]:
input_tensor = tf.convert_to_tensor(onehots, dtype='float64')
noise_tensor = tf.random.uniform(shape=input_tensor.shape, minval=0, maxval=1, dtype='float64')
dequantized_data = tf.add(input_tensor, noise_tensor)

In [ ]:
ds = NumpyDataset(dequantized_data)  # Create a DeepChem dataset
splitter = RandomSplitter()
train, val, test = splitter.train_valid_test_split(dataset=ds, seed=42)
train_idx, val_idx, test_idx = splitter.split(dataset=ds, seed=42)

dim = len(train.X[0])  # length of one-hot encoded vectors
train.X.shape  # 459 samples, N-dimensional one-hot vectors that represent molecules

In [ ]:
base_dist = tfd.MultivariateNormalDiag(loc=np.zeros(dim), scale_diag=np.ones(dim))

if dim % 2 == 0:
    permutation = tf.cast(np.concatenate((np.arange(dim / 2, dim), np.arange(0, dim / 2))),
                                  tf.int32)
else:
    permutation = tf.cast(np.concatenate((np.arange(dim / 2 + 1, dim), np.arange(0, dim / 2))), tf.int32)

In [ ]:
num_layers = 8 # was 8
flow_layers = []

Made = tfb.AutoregressiveNetwork(params=2,
                                 hidden_units=[32], activation='relu') # was 512, 512

for i in range(num_layers):
    flow_layers.append(        
        (tfb.MaskedAutoregressiveFlow(shift_and_log_scale_fn=Made)
    ))

    permutation = tf.cast(np.random.permutation(np.arange(0, dim)), tf.int32)
    
    flow_layers.append(tfb.Permute(permutation=permutation))
    
#     if (i + 1) % int(2) == 0:
#         flow_layers.append(tfb.BatchNormalization())

In [ ]:
%%time
nf = NormalizingFlow(base_distribution=base_dist,
                    flow_layers=flow_layers)

In [ ]:
losses = []
val_losses = []
nfm = NormalizingFlowModel(nf, learning_rate=1e-4, batch_size=64)

In [ ]:
%%time
max_epochs = 600 # was 10

for epoch in range(max_epochs):
  loss = nfm.fit(train, nb_epoch=1, all_losses=losses)
  val_loss = nfm.create_nll(val.X)
  val_losses.append(val_loss.numpy()) # negative log likelihood loss

In [ ]:
f, ax = plt.subplots()
ax.scatter(range(len(losses)), losses, label='train loss')
ax.scatter(range(len(val_losses)), val_losses, label='val loss')
plt.legend(loc='upper right');

In [ ]:
%%time
num_samples = 50
samples_transformed = np.empty((0,dim))#, float64)
counter = 0

while counter < 40:
    print(counter)
    counter += 1
    generated_samples = nfm.flow.sample(num_samples)  # generative modeling
    log_probs = nfm.flow.log_prob(generated_samples)  # probability density estimation
    samples_transformed = np.concatenate((samples_transformed, generated_samples), axis=0)

In [ ]:
samples_transformed.shape

In [ ]:
mols = tf.math.floor(samples_transformed)  # quantize data
mols = tf.clip_by_value(mols, 0, 1)  # Set negative values to 0 and values > 1 to 1
mols_list = mols.numpy().tolist()
import time 

# Add padding characters if needed
for mol in mols_list:
    for i in range(largest_selfie_len):
        row = mol[len(selfies_alphabet) * i: len(selfies_alphabet) * (i + 1)]
    
        for element in enumerate(row):
            if np.isnan(element).any():
                mol[len(selfies_alphabet) * (i+1) - 1] = 1
            else:
                if all(elem == 0 for elem in row):
                    mol[len(selfies_alphabet) * (i+1) - 1] = 1

In [ ]:
mols=sf.batch_flat_hot_to_selfies(mols_list, int_mol)

In [ ]:
valid_count = 0
valid_selfies, invalid_selfies = [], []
for idx, selfies in enumerate(mols):
  try:
    if Chem.MolFromSmiles(sf.decoder(mols[idx]), sanitize=True) is not None:
        valid_count += 1
        valid_selfies.append(selfies)
    else:
      invalid_selfies.append(selfies)
  except Exception:
    pass
print('%.2f' % (valid_count / len(mols)),  '% of generated samples are valid molecules.')

In [ ]:
gen_mols = [Chem.MolFromSmiles(sf.decoder(vs)+'OP(C)(=O)F') for vs in valid_selfies]

gen_mols = [i for i in gen_mols if i]

smiles_generated = [sf.decoder(vs) for vs in valid_selfies] #+'OP(C)(=O)F' for vs in valid_selfies]

from collections import OrderedDict
OrderedDict((x, True) for x in smiles_generated).keys()

In [ ]:
my_range_min = data.idxmax()[0]+1

df_generated = pd.DataFrame(smiles_generated, columns=["Original_SMILES"])#.drop_duplicates()
df_generated['len'] = df_generated['Original_SMILES'].apply(lambda x: len(x))
counter_failed = 0
df_generated['selfies'] = df_generated['Original_SMILES'].apply(preprocess_smiles_NaN)
print(counter_failed)
df_generated['RowID'] = pd.Series(samples_transformed.shape[0])
df_generated['Origin'] = iteration
df_generated.index += my_range_min
new = pd.concat([data, df_generated])#.drop_duplicates(subset=['Original_SMILES'])

In [ ]:
for index, row in new.iterrows():
    try:
        mol = Chem.MolFromSmiles(row['Original_SMILES']+'OP(C)(=O)F')
        qed = QED.default(mol)
        try:
            sas_score = sascorer.calculateScore(mol)
        except:
            sas_score = np.nan 
    except:
        sas_score = np.nan
        qed = np.nan
    
    new.at[index, "QED"] = qed
    new.at[index, "SA_score"] = sas_score

In [ ]:
new['SELFIES_Length'] = new['selfies'].apply(lambda x: sf.len_selfies(x))

fig, ax = plt.subplots()
sns.set_theme(style="whitegrid")

#ax.hist(new_vis['SELFIES_Length'], 15, density=True, histtype='step')#, stepped=True)
ax = sns.histplot(data=new, x="SELFIES_Length", hue="Origin", element="step", bins = 20, multiple="stack")
ax.set_xlabel('Selfie String Length')
plt.axvline(x=16.5)

In [ ]:
fig, ax = plt.subplots()
sns.set_theme(style="whitegrid")

#ax.hist(new_vis['SELFIES_Length'], 15, density=True, histtype='step')#, stepped=True)
ax = sns.histplot(data=new, x="QED", hue="Origin", element="step", bins = 20, multiple="stack")
ax.set_xlabel('QED')
plt.axvline(x=0.712277)

In [ ]:
#fig, ax = plt.subplots()
new_altered = new.copy()
new_altered['Origin'].replace({'1 Iter': '1-5 Iter', 
                               '2 Iter': '1-5 Iter',
                               '3 Iter': '1-5 Iter',
                               '4 Iter': '1-5 Iter',
                               '5 Iter': '1-5 Iter',
                               '6 Iter': '6-10 Iter',
                               '7 Iter': '6-10 Iter',
                               '8 Iter': '6-10 Iter',
                               '9 Iter': '6-10 Iter',
                               '10 Iter': '6-10 Iter',
                              }, inplace=True)

sns.jointplot(data=new_altered, x="QED", y="SELFIES_Length", hue="Origin")#, xlim=(0,0.8), ylim=(0,30))
sns.reset_orig
plt.show()
#ax.set_xlabel('Calculated QED for the sample molecule')
#ax.set_ylim(bottom=0.1, top=0.80)
#ax.set_xlim(left=0.1, right=0.80)
#ax.legend(bbox_to_anchor=(1.0, 1.00))
#plt.axvline(x=0, y=1)
#plt.axhline(y=0.712277)

In [ ]:
symbol_to_int = dict((c, i) for i, c in enumerate(selfies_alphabet))

analysis_input=sf.batch_selfies_to_flat_hot(selfies_list, symbol_to_int, largest_selfie_len)
analysis_input_results = np.zeros((len(analysis_input[0]),), dtype=int).tolist()

for i in analysis_input: #range(0, 2):#len(analysis_input)):
    #print(i)
    summa = []
    for k in range(0, len(analysis_input[0])):
        #print(analysis_input_results[k] , 'and',  i[k])
        summa.append(analysis_input_results[k] + i[k])
    analysis_input_results = summa

def split_list(lst, chunk_size):
    return [lst[i:i + chunk_size] for i in range(0, len(lst), chunk_size)]

input_analysis = split_list(analysis_input_results, len(selfies_alphabet))
# symbol_to_int # = x 

In [ ]:
symbol_to_int = dict((c, i) for i, c in enumerate(selfies_alphabet))

analysis_output=sf.batch_selfies_to_flat_hot(mols, symbol_to_int, largest_selfie_len)
analysis_output_results = np.zeros((len(analysis_output[0]),), dtype=int).tolist()

for i in analysis_output: #range(0, 2):#len(analysis_input)):
    #print(i)
    summa = []
    for k in range(0, len(analysis_output[0])):
        summa.append(analysis_output_results[k] + i[k])
    analysis_output_results = summa

output_analysis = split_list(analysis_output_results, len(selfies_alphabet))

histogram_inp = dict(list(enumerate(input_analysis)))
histogram_out = dict(list(enumerate(output_analysis)))

histogram_inp['Selfie'] = selfies_alphabet
histogram_out['Selfie'] = selfies_alphabet

In [ ]:
fig, axes = plt.subplots(5, 4, figsize=(24, 30))
plt.subplots_adjust(left=0.1,
                    bottom=0.05, 
                    right=0.99, 
                    top=0.97,
                    wspace=0.3, 
                    hspace=0.6
                   )

count=0
position=[axes[0, 0], axes[0, 1], axes[0, 2], axes[0, 3],
          axes[1, 0], axes[1, 1], axes[1, 2], axes[1, 3],
          axes[2, 0], axes[2, 1], axes[2, 2], axes[2, 3],
          axes[3, 0], axes[3, 1], axes[3, 2], axes[3, 3],
          axes[4, 0], axes[4, 1], axes[4, 2], axes[4, 3],
         ]

names =     ["0th", "1st", "2nd", "3rd",
             "4th", "5th", "6th", "7th",
             "8th", "9th", "10th", "11th",
             "12th", "13th", "14th", "15th",
             "16th", "17th", "18th", "19th",
            ]

for ax, title in zip(axes.flat, names):
    ax.set_title(title, fontsize=16)

for i in range(0, len(position)):
    data_vis = pd.DataFrame()
    
    data_vis['Selfie'] = selfies_alphabet
    data_vis['Input'] = histogram_inp[i]
    data_vis['Output'] = histogram_out[i]
    
    data_vis.plot(ax=position[count], kind='bar', x='Selfie', y=['Input', 'Output'], rot = 90)

    position[count].set_yscale('log')
#    position[count].set_ylim(top=200)
    
    # deleting the duplicate x and y labels
    axes[0, 0].set_xlabel('', fontsize=20)
    axes[0, 1].set_xlabel('', fontsize=20)
    axes[0, 2].set_xlabel('', fontsize=20)
    axes[0, 3].set_xlabel('', fontsize=20)
#    axes[1, 0].set_xlabel('', fontsize=20)
#    axes[1, 1].set_xlabel('', fontsize=20)
#    axes[1, 2].set_xlabel('', fontsize=20)
#    axes[1, 3].set_xlabel('', fontsize=20)
    axes[2, 0].set_xlabel('', fontsize=20)
    axes[2, 1].set_xlabel('', fontsize=20)
    axes[2, 2].set_xlabel('', fontsize=20)
    axes[2, 3].set_xlabel('', fontsize=20)
    
    count=count+1

fig.savefig('Distributions-in-out.png')

In [ ]:
new_samples = new.loc[(new['QED'] > 0.712277) | (new['SELFIES_Length'] > 10) ].loc[(new['Origin'] == iteration)].drop_duplicates(subset=['Original_SMILES']).dropna(subset=['QED'])
saving = pd.concat([data, new_samples]).drop_duplicates(subset=['Original_SMILES']).dropna(subset=['QED'])

In [ ]:
new_samples.describe()